In [1]:
import find_eoms
import find_A_theta

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
import numpy as np

# Number of samples
q1_low  = -torch.pi
q1_high =  torch.pi
q2_low  = -torch.pi
q2_high =  torch.pi

n_samples = 100000

# Generate uniformly distributed points for q1 and q2
#q1 = torch.random.uniform(low=-10, high=10, size=n_samples)
#q2 = torch.random.uniform(low=-10, high=10, size=n_samples)
q1 = torch.linspace(q1_low, q1_high, n_samples)
q2 = torch.linspace(q2_low, q2_high, n_samples)

idx = torch.randperm(q2.shape[0])

q2 = q2[idx]

# Stack q1 and q2 to get the 2D coordinates
points = torch.stack([q1, q2], axis=1)  # Shape will be (1000, 2)

In [3]:
# Use TensorDataset to create the dataset
dataset = TensorDataset(points)

# Create the DataLoader with batch size and shuffling
batch_size = 128
dataloader = DataLoader(dataset, 
                        batch_size=batch_size, 
                        shuffle=True,
                        num_workers=0,
                        pin_memory=True)

print(dataloader)

# Example usage: iterate through the DataLoader
for batch in dataloader:
    #print(batch)
    break  # Just to show one batch

/home/kian/anaconda3/envs/thesis/lib/python3.12/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
dataiter = iter(dataloader)
coordinates = next(dataiter)
print(coordinates)


[tensor([[ 1.0257e+00,  1.3773e+00],
        [ 2.5319e+00, -1.1094e+00],
        [ 6.7573e-01,  7.9637e-01],
        [ 2.8588e-03, -2.2661e+00],
        [-7.3385e-01, -2.6372e+00],
        [-1.3899e+00,  1.1333e+00],
        [-1.8960e-01, -1.7243e+00],
        [-2.2119e+00, -2.7771e+00],
        [-4.1567e-01,  1.1928e+00],
        [ 3.0735e-01, -4.6339e-02],
        [-1.0799e+00, -2.2072e+00],
        [-1.9091e+00,  1.5539e+00],
        [ 2.8532e+00,  2.4263e+00],
        [ 1.0801e+00,  3.4944e-01],
        [ 2.6123e-01,  1.3061e+00],
        [ 2.3049e+00,  2.4553e+00],
        [ 1.9693e+00, -1.6841e+00],
        [ 5.7891e-01,  3.0293e+00],
        [-1.8993e+00,  1.2567e+00],
        [-2.2707e+00, -1.0969e+00],
        [-1.4710e+00,  2.2529e-01],
        [-2.1015e+00, -6.1083e-01],
        [-1.4463e+00,  2.4419e+00],
        [-1.8631e+00,  9.2756e-01],
        [ 2.0251e+00, -2.8927e+00],
        [-1.9320e+00,  9.5213e-01],
        [-2.0729e+00, -3.3329e-01],
        [ 2.7037e+00,  3.04

In [5]:
class SinCosLayer(nn.Module):
    def __init__(self):
        super(SinCosLayer, self).__init__()

    def forward(self, x):
        # Apply sin() and cos() to both coordinates
        x_sin = torch.sin(x)
        x_cos = torch.cos(x)
        x_sin_cos_shape = (x.shape[0], x.shape[1]*2)
        x_sin_cos = torch.empty(x_sin_cos_shape, dtype=x_sin.dtype, device=x.device)
        x_sin_cos[:,0::2] = x_sin
        x_sin_cos[:,1::2] = x_cos
        return x_sin_cos
    
# Custom layer to reverse the interleaved sin() and cos() back to original coordinates
class InverseSinCosLayer(nn.Module):
    def __init__(self):
        super(InverseSinCosLayer, self).__init__()

    def forward(self, x):
        # x contains interleaved sin() and cos() values
        # Assuming input is of shape (batch_size, 4) for 2D coordinates
        sin_vals = x[:, 0::2]  # Extract sin values
        cos_vals = x[:, 1::2]  # Extract cos values

        # Use atan2 to recover the original angles from sin and cos
        original_coords = torch.atan2(sin_vals, cos_vals)
        return original_coords
    



class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            #SinCosLayer(),
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2)#,
            #InverseSinCosLayer()
        )
        
        self.decoder = nn.Sequential(
            #SinCosLayer(),
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 2)#,
            #InverseSinCosLayer()
        )
    
    def forward(self, q):
        theta = self.encoder(q)
        q_hat = self.decoder(theta)
        return(theta, q_hat)

In [6]:
%%time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder().to(device)  # Move model to GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-7)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
#scheduler.get_last_lr()


num_epochs = 50
outputs = []
for epoch in range(num_epochs):
    for (q) in dataloader:
        q = q[0].to(device)
        theta, q_hat = model(q)
        loss_reconstruction = criterion(q_hat, q)
        loss_theta = criterion(theta, q)
        loss = loss_reconstruction + loss_theta
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    #print(scheduler.get_last_lr())
        
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.7f}')
    outputs.append((epoch, q, q_hat, theta))

Epoch:1, Loss:0.7555522
Epoch:2, Loss:0.0407802
Epoch:3, Loss:0.0175672
Epoch:4, Loss:0.0065178
Epoch:5, Loss:0.0028582
Epoch:6, Loss:0.0017482
Epoch:7, Loss:0.0011013
Epoch:8, Loss:0.0005336
Epoch:9, Loss:0.0007024
Epoch:10, Loss:0.0004704
Epoch:11, Loss:0.0004571
Epoch:12, Loss:0.0002910
Epoch:13, Loss:0.0001594
Epoch:14, Loss:0.0001936
Epoch:15, Loss:0.0001759
Epoch:16, Loss:0.0001628
Epoch:17, Loss:0.0001412
Epoch:18, Loss:0.0000842
Epoch:19, Loss:0.0000915
Epoch:20, Loss:0.0000599
Epoch:21, Loss:0.0000828
Epoch:22, Loss:0.0000763
Epoch:23, Loss:0.0000529
Epoch:24, Loss:0.0000548
Epoch:25, Loss:0.0000389
Epoch:26, Loss:0.0000645
Epoch:27, Loss:0.0000519
Epoch:28, Loss:0.0000501
Epoch:29, Loss:0.0000450
Epoch:30, Loss:0.0000410
Epoch:31, Loss:0.0000389
Epoch:32, Loss:0.0000248
Epoch:33, Loss:0.0000460
Epoch:34, Loss:0.0000282
Epoch:35, Loss:0.0000316
Epoch:36, Loss:0.0000311
Epoch:37, Loss:0.0000308
Epoch:38, Loss:0.0000287
Epoch:39, Loss:0.0000124
Epoch:40, Loss:0.0000303
Epoch:41,

In [7]:
print("q\n", outputs[-1][1][0:4])
print("q_hat\n", outputs[-1][2][0:4])
print("theta\n", outputs[-1][3][0:4])
print("\n\n")
print("q_hat - q\n", outputs[-1][2][0:4] - outputs[-1][1][0:4])
print("theta - q\n", outputs[-1][3][0:4] - outputs[-1][1][0:4])

q
 tensor([[-0.6228,  0.8825],
        [-1.3864, -0.4348],
        [-1.9743,  1.1098],
        [ 3.0074,  1.5706]])
q_hat
 tensor([[-0.6230,  0.8825],
        [-1.3875, -0.4380],
        [-1.9801,  1.1103],
        [ 3.0096,  1.5705]], grad_fn=<SliceBackward0>)
theta
 tensor([[-0.6240,  0.8824],
        [-1.3838, -0.4336],
        [-1.9780,  1.1049],
        [ 3.0090,  1.5712]], grad_fn=<SliceBackward0>)



q_hat - q
 tensor([[-2.1225e-04, -4.2975e-05],
        [-1.1698e-03, -3.2139e-03],
        [-5.8438e-03,  4.2236e-04],
        [ 2.1656e-03, -9.5367e-05]], grad_fn=<SubBackward0>)
theta - q
 tensor([[-1.1581e-03, -7.1645e-05],
        [ 2.5474e-03,  1.1711e-03],
        [-3.6832e-03, -4.9821e-03],
        [ 1.5061e-03,  5.9140e-04]], grad_fn=<SubBackward0>)


In [8]:
for param in model.parameters():
  print(param.data)

tensor([[-4.8425e-01, -1.2549e-01],
        [-4.2729e-01,  3.5862e-01],
        [-2.8967e-01, -5.6129e-01],
        [ 2.5815e-01, -5.6163e-01],
        [ 3.5365e-01,  4.6801e-01],
        [-1.6979e-01,  3.7043e-01],
        [ 6.6216e-01, -5.5432e-01],
        [-5.3130e-01, -6.1119e-01],
        [-6.5800e-01,  1.2260e-01],
        [ 1.2917e-01,  5.6369e-01],
        [-2.9386e-01, -3.8337e-01],
        [-3.7425e-01,  3.6169e-01],
        [-4.7829e-02, -2.0564e-01],
        [ 5.9787e-01,  6.9688e-01],
        [ 2.8378e-07, -1.6153e-07],
        [ 7.5408e-02, -7.0950e-02]])
tensor([ 0.4115,  0.1610,  0.3729,  0.2973,  0.1830, -0.1983, -0.2560, -0.6250,
         0.6016,  0.5178, -0.1427,  0.8024,  0.7671,  0.7279, -0.0009,  0.4485])
tensor([[ 2.8608e-01,  9.5503e-03,  1.7226e-02,  ...,  2.8224e-01,
          5.6570e-08,  9.5434e-02],
        [ 2.2408e-01,  1.4952e-01,  1.8205e-01,  ...,  8.1742e-02,
          2.1727e-07, -2.6455e-01],
        [ 2.0743e-07, -3.4731e-08, -1.8416e-08,  ..., -2

In [9]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-4.8425e-01, -1.2549e-01],
        [-4.2729e-01,  3.5862e-01],
        [-2.8967e-01, -5.6129e-01],
        [ 2.5815e-01, -5.6163e-01],
        [ 3.5365e-01,  4.6801e-01],
        [-1.6979e-01,  3.7043e-01],
        [ 6.6216e-01, -5.5432e-01],
        [-5.3130e-01, -6.1119e-01],
        [-6.5800e-01,  1.2260e-01],
        [ 1.2917e-01,  5.6369e-01],
        [-2.9386e-01, -3.8337e-01],
        [-3.7425e-01,  3.6169e-01],
        [-4.7829e-02, -2.0564e-01],
        [ 5.9787e-01,  6.9688e-01],
        [ 2.8378e-07, -1.6153e-07],
        [ 7.5408e-02, -7.0950e-02]], requires_grad=True)
Parameter containing:
tensor([ 0.4115,  0.1610,  0.3729,  0.2973,  0.1830, -0.1983, -0.2560, -0.6250,
         0.6016,  0.5178, -0.1427,  0.8024,  0.7671,  0.7279, -0.0009,  0.4485],
       requires_grad=True)
Parameter containing:
tensor([[ 2.8608e-01,  9.5503e-03,  1.7226e-02,  ...,  2.8224e-01,
          5.6570e-08,  9.5434e-02],
        [ 2.2408e-01,  1.4952e-01,  1.8205e-01